In [55]:
reload_lamb()

Simple type inference still works

In [56]:
%te P_<e,t>(x_e) & P_<n,t>(n_n) # should fail because no principle type for P

ERROR (parsing): Parsing of typed expression failed with exception:
ERROR (parsing): Type mismatch: 'P_<e,t>'/<e,t> and type <n,t> conflict (mode: Failed to unify types across distinct instances of term)


Inference from type variables to simple types

In [57]:
%te L x_e : Q_<X,X>(x_X)

(λ x_e: Q_<e,e>(x_e))

In [58]:
%te Q_<X,X>(x_e)

Q_<e,e>(x_e)

In [59]:
%te p_<e,t>(x_e) & p_<X,t>(n_X) # variables of the same name should have the same type

(p_<e,t>(x_e) & p_<e,t>(n_e))

Application-related inference

In [60]:
%te Q_X(x_e)

Q_<e,X>(x_e)

In [61]:
%te (L x_X : P_<X,X>(x))(x_e)

((λ x_e: P_<e,e>(x_e)))(x_e)

In [62]:
%te (f_<t,t>(p_<X,X>(Q_<X,X>(x_X))))

f_<t,t>(p_<t,t>(Q_<t,t>(x_t)))

In [63]:
%te (f_<<X,Y>,<X,Y>>(g_<X,X>))

f_<<Y,Y>,<Y,Y>>(g_<Y,Y>)

In [64]:
%te (L f_<X,Y> : L x_X: y_Y)(g_<Z,Z>)

((λ f_<Y,Y>: (λ x_Y: y_Y)))(g_<Y,Y>)

In [65]:
%te p_X(q_X'(a_X''(b_X'''(c_X''''(d_e)))))

p_<X''',X4>(q_<X'',X'''>(a_<X',X''>(b_<X,X'>(c_<e,X>(d_e)))))

In [66]:
%te L x_X : x(y_<e,t>)

(λ x_<<e,t>,X>: x_<<e,t>,X>(y_<e,t>))

In [67]:
%te (L g_<Y,t> : L x_X : g(x)) (L x_X10 : P_<Z,t>(x)) # application involving two LFuns

((λ g_<Y,t>: (λ x_Y: g_<Y,t>(x_Y))))((λ x_Y: P_<Y,t>(x_Y)))

Interaction with other operators

In [68]:
%te P_<e,t>(x_X) & Q_<X,Y>(x_X)

(P_<e,t>(x_e) & Q_<e,t>(x_e))

In [69]:
%te p_t & Q_<X,X>(x_X)

(p_t & Q_<t,t>(x_t))

In [70]:
x = %te Iota x_X: P(x)
x.try_adjust_type(tp("e"))

INFO (meta): Coerced guessed type t for 'P_t' into <X,t>, to match argument 'x_X'


(ι x_e: P_<e,t>(x_e))

In [71]:
x.try_adjust_type(tp("Y"))

(ι x_Y: P_<Y,t>(x_Y))

In [72]:
%te (L x_X : P_<e,t>(x))(Iota x_Y: Q_<Y,t>(x))

((λ x_e: P_<e,t>(x_e)))((ι x_e: Q_<e,t>(x_e)))

Inference from function body to variable

In [73]:
%te L x_X : x_e

(λ x_e: x_e)

In [74]:
%te L x_e : x_X

(λ x_e: x_e)

In [75]:
%te L x_X : z_X(y_e)

(λ x_<e,X>: z_<e,X>(y_e))

In [76]:
%te L x_X : x_X & x_X

(λ x_t: (x_t & x_t))

In [77]:
%te L x_X : g_<Y,t>(x)

(λ x_Y: g_<Y,t>(x_Y))

In [78]:
%te L g_<Y,t> : L x_X : g(x) # across multiple LFuns

(λ g_<Y,t>: (λ x_Y: g_<Y,t>(x_Y)))

In [79]:
%te L x_X : (L y_Y : (L a_Z : a(z_e))(y))(x) & p_X'

(λ x_<e,t>: (((λ y_<e,t>: ((λ a_<e,t>: a_<e,t>(z_e)))(y_<e,t>)))(x_<e,t>) & p_t))

In [80]:
%te L a_X : L b_X' : a(b) # application should force functional type

(λ a_<X',X>: (λ b_X': a_<X',X>(b_X')))

In [81]:
%te L a_X : L c_X'' : a((L b_X' : b)(c)) # X'' = X', X = <X', ?>

(λ a_<X',X>: (λ c_X': a_<X',X>(((λ b_X': b_X'))(c_X'))))

In [82]:
%te L d_X22 : L c_X' : L b_X'' : L a_X''' : L q_X'''' : L p_X''''' : p(q(a(b(c(d)))))

(λ d_X5: (λ c_<X5,X>: (λ b_<X,X'>: (λ a_<X',X''>: (λ q_<X'',X'''>: (λ p_<X''',X4>: p_<X''',X4>(q_<X'',X'''>(a_<X',X''>(b_<X,X'>(c_<X5,X>(d_X5)))))))))))

Inference in function construction, failure cases.  

The following should fail because there is no consistent resolution of type variable `X`.

In [84]:
%te L x_X : (L y_<Y,Z> : y(z_e))(x) & p_X # X = <Y,Z> (appl), Y = <e,?> (appl), Z = t (conjunction), X=<e,t> (unification), X = t (conjunction)

ERROR (parsing): Parsing of typed expression failed with exception:
ERROR (parsing): Type mismatch: type <e,t> and type t conflict (mode: Failed to unify type variable X across contexts)


Various things in one

In [85]:
%te L x_X : P_<Y,t>(A_Y) & Q_<Y,t>(x_e) # Y=X (appl), X=e (from var), inference to type vars not shared on bound variables

(λ x_e: (P_<e,t>(A_e) & Q_<e,t>(x_e)))

In [86]:
%te (L x_X : P_<Y,t>(A_Y) & Q_<X,t>(x_e))((L x_Y : x)(A_Y))

((λ x_e: (P_<e,t>(A_e) & Q_<e,t>(x_e))))(((λ x_e: x_e))(A_e))

In [87]:
# ex of geach rule: complex inference to parts of function
%te (λ g_<Y,Z>: (λ f_<X,Y>: (λ x_X: g_<Y,Z>(f_<X,Y>(x_X)))))(L x_e : L y_e : Saw(y,x))

INFO (meta): Coerced guessed type t for 'Saw_t' into <(e,e),t>, to match argument '(y_e, x_e)'


((λ g_<e,<e,t>>: (λ f_<X,e>: (λ x_X: g_<e,<e,t>>(f_<X,e>(x_X))))))((λ x_e: (λ y_e: Saw_<(e,e),t>(y_e, x_e))))